# ANALYZING EDUCATIVE COURSES DATA

## Background  
This analysis project focuses on the educational technology company, Educative, known for its online platform that offers a diverse range of courses to empower individuals seeking to enhance their skills in both professional and academic domains.
Analysis of data from the company will be in-line with <b>Cross Industry Standard for Data Mining (CRISP-DM)


# Planned Steps
* Create a extraction file location 
* Unzip the CSV 
* Merge CSVs into one dataframe and save that in excel as the first sheet
* Do EDA to check vitals (Nulls, Duplicates, Shape, etc)

* <b>Clean Data</b>
    * Drop the URL Column as it wont be in use
    * Remove Duplicated rows based on the courses_id column
    * Remove suffix of "subject: " from the subject columns
    * Create 2 columns 'Free_or_Paid' and 'Revenue'
    * Split the publised_timestamp columns into publised_date(Change data format to dd/mm/yyyy) and published_time
    * Create a time_of_day (Morning/Afternoon/ Evening) column based on the published_time
    * Change columns names and reorder columns
    * Change the column datatype  
    * Do EDA on the new dataframe created
    * Find the top 20 courses per subscribers and add some key columns like Free_or_Paid/Levels/publised_date  
      
 
* <b>Create a final excel sheet with 3 sheets to hold , the original data/cleaned data/ summarydata <b/>   
* Create Pivot Tables and Visualize some Data
* Create a report using PowerBI

In [1]:
import os
import zipfile
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Unzipping the dataset.zip file into the current working directory
zip_path="dataset.zip"
ext_path=os.getcwd()

#creating the extracted folder
if not os.path.exists(ext_path):
    os.makedirs(ext_path)
#Unziping file

with zipfile.ZipFile(zip_path,'r') as zipping:
    zipping.extractall(ext_path)

In [ ]:
#Merging Files
all_files=glob.glob("Data Sheet Educative Courses Export\\Data_Sheet*.csv")
print(f'List of all the files \n :{all_files}\n\n')

df_list=[pd.read_csv(file) for file in all_files] #Created a list of dataframes form the all_files list
all_csvs=pd.concat(df_list,ignore_index=True)
all_csvs

## Basic Exploratory Data Analysis

In [ ]:
print(all_csvs.shape)

In [ ]:
print (all_csvs.info())

In [ ]:
all_csvs.isnull().sum()
#Checking the number of nulls in the duplicate of the original data
print(f'List of the null columns in original data\n\n{all_csvs.isnull().sum()}\n')
print(f'\nNumber of duplicated records in original data  : {all_csvs.duplicated().sum()}')

In [ ]:
#Creating duplicated dataset to work on
dup_data=all_csvs.copy()
dup_data

In [ ]:
dup_data.shape

In [ ]:
dup_data.info()

In [ ]:
#Checking the number of nulls in the duplicate of the original data
print(f'List of the null columns before cleaning \n\n{dup_data.isnull().sum()}\n')
print(f'\nNumber of duplicated records before cleaning : {dup_data.duplicated().sum()}')

In [ ]:
#Dropping the URL column as it won't be of any use to the analysis
dup_data=dup_data.drop(columns='url')
dup_data.head()

In [ ]:
#Dropping rows if they have no data in the course_id column and resetting indexes after
dup_data=dup_data.dropna(subset='course_id')
dup_data=dup_data.reset_index(drop=True)
dup_data

In [ ]:
#Checking the new shape after a little cleaning
dup_data.shape

In [ ]:
#Checking the number of nulls in the duplicate of the original data
print(f'List of the null columns after cleaning \n\n{dup_data.isnull().sum()}')
print(f'\nNumber of duplicated records after cleaning : {dup_data.duplicated().sum()}')

In [ ]:
#Descriptive Statistics on the numerical column
dup_data.describe()

In [ ]:
#Descriptive Statistics on the numerical column
dup_data.describe(include='object')

In [ ]:
#Creating of the Revenue column and free_or_paid
dup_data['revenue']=dup_data['price']*dup_data['num_subscribers']
dup_data['Free_or_Paid']=dup_data['price'].apply(lambda x: 'Free' if x ==0 else 'Paid')
dup_data

In [ ]:
#Cleaning the Subject column by taking off the 'Subject : 'prefix for some records
dup_data['subject']=dup_data['subject'].str.lstrip('Subject:')
dup_data

In [ ]:
#Splitting the published_timestamp to published_date and published_time
dup_data[['published_date','published_time']]=dup_data['published_timestamp'].str.split('T',expand=True) #T is the delimiter
dup_data['published_time']=dup_data['published_time'].str.rstrip('Z') 
dup_data=dup_data.drop(columns='published_timestamp')
dup_data.head()

In [ ]:
#Converting from yyyy/mm/dd to dd/mm/yyyy in the published_date column
dup_data['published_date']=pd.to_datetime(dup_data['published_date']).dt.strftime('%d/%m/%Y')
dup_data

In [ ]:
#Converting the published_time to a time datatype to assit you create the time_of_day
#Categorizing the published time into time of the day Morning (times which are less than 12pm), Afternoon(less than 6pm),Evening
#12:00:00 == 43200secs // 18:00:00 == 64800secs (Morning <12pm, Afternoon <6pm)
dup_data['times_in_seconds']=pd.to_timedelta(dup_data['published_time']).dt.total_seconds().astype(int)
dup_data['time_of_day']=dup_data['times_in_seconds'].apply(lambda x:'Morning' if x< 43200 else('Afternoon' if x < 64800 else 'Evening'))
dup_data=dup_data.drop(columns='times_in_seconds')
dup_data

In [ ]:
dup_data.info()

In [ ]:
print(f'original columns : {all_csvs.columns}')
print(f'duplicated data columns : {dup_data.columns}')

In [ ]:
#Changing Column Names
new_names=['Course_ID', 'Course_Title', 'Price', 'Number_of_Subscribers', 'Number_of_Reviews','Number_of_Lectures', 
           'Level', 'Rating', 'Content_Duration', 'Subject','Revenue', 'Free_or_Paid', 'Published_Date', 'Published_Time',
       'Time_of_Day']
dup_data.columns=new_names
dup_data

In [ ]:
#Reordering the columns
reorder_cols=['Course_ID', 'Course_Title', 'Free_or_Paid','Price', 'Number_of_Subscribers', 'Revenue','Number_of_Reviews',
              'Number_of_Lectures', 'Level', 'Rating', 'Content_Duration', 'Subject',  'Published_Date', 'Published_Time',
              'Time_of_Day']
dup_data=dup_data.reindex(columns=reorder_cols)
dup_data.head()

In [ ]:
#Changing the datatypes
dup_data['Course_ID']=dup_data['Course_ID'].astype('object')
dup_data['Free_or_Paid']=dup_data['Free_or_Paid'].astype('category')
dup_data['Subject']=dup_data['Subject'].astype('category')
dup_data['Level']=dup_data['Level'].astype('category')
dup_data['Number_of_Subscribers']=dup_data['Number_of_Subscribers'].astype('int')
dup_data['Number_of_Reviews']=dup_data['Number_of_Reviews'].astype('int')
dup_data['Number_of_Lectures']=dup_data['Number_of_Lectures'].astype('int')
dup_data['Number_of_Subscribers']=dup_data['Number_of_Subscribers'].astype('int')
dup_data['Time_of_Day']=dup_data['Time_of_Day'].astype('category')

In [ ]:
dup_data.info()

In [ ]:
dup_data.head()

In [ ]:
dup_data.corr(numeric_only=True)

In [ ]:
cleaned_data=dup_data.copy()

In [ ]:
#Summary data of the top 20 subjects based on the subscribers
target_cols=['Course_Title','Number_of_Subscribers','Level','Free_or_Paid','Content_Duration','Published_Date']
top_20_courses=dup_data[target_cols].sort_values(by='Number_of_Subscribers',ascending=False).copy()
top_20_courses=top_20_courses.sort_values(by='Number_of_Subscribers',ascending=False).head(20)
top_20_courses=top_20_courses.reset_index(drop=True)
top_20_courses

In [ ]:
#Saving the 3 data into one excel
with pd.ExcelWriter('Educative Data.xlsx') as master:
    all_csvs.to_excel(master, sheet_name='Main_Data', index=False)
    cleaned_data.to_excel(master, sheet_name='Cleaned_Data', index=False)
    top_20_courses.to_excel(master, sheet_name='Top_20_Subs_Courses', index=False)

# Data Visualization

* <b>Visualizations</b>
    * Create the pivot table for the following and visualize  
        ●Total number of subscribers for each subject   
        ●Average subscriber count per subject  
        ●Average cost per subject at each level  
        ●Average content duration per subject  
        ●Average rating per subject for each level  
        ●Revenue Generated per each subject over the years
  
* Save visuals in a PDF format after.                

In [ ]:
#Creating a duplicate of the cleaned data for my pivot table
pivot_data=cleaned_data.copy()
pivot_data['Published_Date'] = pd.to_datetime(pivot_data['Published_Date'], format='%d/%m/%Y')

# Create a new column for just the year
pivot_data['Published_Year'] = pivot_data['Published_Date'].dt.year
pivot_data.head()


In [ ]:
#Pivot Table for the Total Number of Subscribers Per Subject
subject_subs=pd.pivot_table(data=pivot_data,index='Subject',values='Number_of_Subscribers',aggfunc='sum')
subject_subs

In [ ]:
custom_palette = sns.color_palette(["#607C3C", "#ABC32F", "#809C13", "#B5E550"])
#custom_palette = sns.color_palette(["#556B2F", "#8FBC8F", "#C0D9AF", "#E0EEE0"])
explode_list= [0.1,0,0.1,0.1]
plt.pie(subject_subs["Number_of_Subscribers"],
        autopct = '%.1f%%',labels=subject_subs.index,
        shadow=True, colors=custom_palette,
       startangle=45);
plt.title('Ratio of Subscribers Per Subject',
          fontsize=16, fontweight='bold')
plt.savefig('subs_per_subject_ratio_pie_chart.png')
plt.show()

In [ ]:
#Average subscriber count per subject
avg_sub_subject=round((pd.pivot_table(data=pivot_data, index='Subject',values='Number_of_Subscribers',aggfunc='mean')),2)
avg_sub_subject=avg_sub_subject.sort_values(by='Number_of_Subscribers',ascending=False)
avg_sub_subject

In [ ]:

custom_palette = sns.color_palette(["#607C3C", "#ABC32F", "#809C13", "#B5E550"])
ax=sns.barplot(data=avg_sub_subject,x=avg_sub_subject.index,y=avg_sub_subject['Number_of_Subscribers'],
            palette=custom_palette,order=avg_sub_subject.index)

for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height),
                ha='center', va='center', fontsize=12, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.xticks(fontsize=8,fontweight='bold')  
plt.yticks([])
plt.xlabel(None)  # X-axis label
plt.ylabel('Avg Subscribers Number', fontsize=12,labelpad=15)  # Y-axis label
plt.title('Average Number of Subscribers Per Subject', fontsize=16, fontweight='bold',pad=20) 
plt.savefig('avg_subs_num_per_subject_column_chart.png')
plt.show()

In [ ]:
#Average content duration per subject
avg_dur_subj=round((pd.pivot_table(data=pivot_data, index='Subject',values='Content_Duration',aggfunc='mean')),1)
avg_dur_subj

In [ ]:

custom_palette = sns.color_palette(["#607C3C", "#ABC32F", "#809C13", "#B5E550"])
ax=sns.barplot(data=avg_dur_subj,x=avg_dur_subj.index,y=avg_dur_subj['Content_Duration'],
            palette=custom_palette,order=avg_dur_subj.index)


    
plt.xticks(fontsize=8, fontweight='bold') 
plt.xlabel(None)  # X-axis label
plt.ylabel('Average Rating', fontsize=14,labelpad=15)  # Y-axis label
plt.title('Average Rating Per Subject', fontsize=16, fontweight='bold',pad=20) 
plt.savefig('avg_rat_per_subject_column_chart.png')
plt.show()

In [ ]:
# Average cost per subject at each level
avg_sub_subjlvl=round((pd.pivot_table(data=pivot_data, index='Subject', columns='Level',values='Price',aggfunc='mean')),2)
avg_sub_subjlvl

In [ ]:
custom_palette =["#607C3C",  "#809C13", "#ABC32F", "#B5E550"]
sns.set_palette(custom_palette)
sns.catplot(data=cleaned_data,x='Subject',y='Price',hue='Level', kind='bar',aspect=2)
plt.xticks(fontsize=8, fontweight='bold')
plt.xlabel(None)
plt.ylabel('Rating', fontsize=14)
plt.title(' Average Price Per Subject For Each Level',  fontsize=16, fontweight='bold',pad=20)
plt.savefig('avg_price_per_subjectlevel_column_chart.png')
plt.show()

In [ ]:
#Average rating per subject for each level
avg_rat_subjlvl=round((pd.pivot_table(data=pivot_data, index='Subject', columns='Level',values='Rating',aggfunc='mean')),2)
avg_rat_subjlvl

In [ ]:
custom_palette =["#607C3C",  "#809C13", "#ABC32F", "#B5E550"]
sns.set_palette(custom_palette)
sns.catplot(data=cleaned_data,x='Subject',y='Rating',hue='Level', kind='bar',aspect=2)
plt.xticks(fontsize=8, fontweight='bold')
plt.xlabel(None)
plt.ylabel('Rating', fontsize=14)
plt.title(' Average Rating Per Subject For Each Level',  fontsize=16, fontweight='bold',pad=20)
plt.savefig('avg_rating_per_subjectlevel_column_chart.png')
plt.show()

In [ ]:
#Creating Pivot Table of the data
yoy_subj_revenue=(pd.pivot_table(data=pivot_data, index='Published_Year',values='Revenue',aggfunc="sum"))

yoy_subj_revenue

In [ ]:
sns.relplot(data=yoy_subj_revenue, kind="line",markers=True,legend=False,aspect=1.5)
plt.xticks(fontsize=8, fontweight='bold')
plt.yticks(([]))
plt.xlabel(None)
plt.ylabel('Revenue', fontsize=14,labelpad=15,fontweight='bold')
plt.title('Revenue Generated Per Year',  fontsize=16, fontweight='bold',pad=20)
plt.savefig('yoy_revenue_line_graph.png')
plt.show()